In [0]:


%pip install pdfplumber pandas

import pdfplumber
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_extract, col
 
dbutils.library.restartPython()


In [0]:
%restart_python

In [0]:
import os
import pdfplumber
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import regexp_extract, col

# Initialize Spark session
spark = SparkSession.builder.getOrCreate()

# Define directory path
pdf_dir = "/Volumes/databricks_catalog/invoice_schema/pdf/PDF_Invoice_Folder/"

# Get list of all PDF files in the directory
pdf_files = [f for f in os.listdir(pdf_dir) if f.lower().endswith(".pdf")]

# Extract text from each PDF file
data = []
for file_name in pdf_files:
    file_path = os.path.join(pdf_dir, file_name)
    with pdfplumber.open(file_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text() or ""
        data.append(Row(file_name=file_name, raw_text=text))

# Create a Spark DataFrame from extracted data
pdf_df = spark.createDataFrame(data)

pdf_df = pdf_df.filter(col("file_name") != "invoice_Chuck Magee_45227.pdf")

# View the raw data
display(pdf_df)

# Extract structured fields using regex
parsed_df = (
    pdf_df
    .withColumn("row_id", regexp_extract("raw_text", r"#\s*(\d+)", 1))
    .withColumn("order_date", regexp_extract("raw_text", r"Date:\s*(\w+\s+\d+\s+\d+)", 1))
    .withColumn("ship_mode", regexp_extract("raw_text", r"Ship Mode:\s*(.+?)\s", 1))
    .withColumn("customer_name", regexp_extract("raw_text", r"Class\s*([\w\s]+?)\s+\d{5},", 1))
    .withColumn("address", regexp_extract("raw_text", r"Class\s+[A-Za-z]+\s+[A-Za-z]+\s+((?:\d{5},\s+)?[A-Za-z\s]+,\s*[A-Za-z\s]+,\s*[A-Za-z\s]+)(?=\s+Balance)", 1))
    .withColumn("postal_code", regexp_extract("raw_text", r"Ship Mode:.*?\n.*?\b(\d{5})\b", 1))
    .withColumn("product_name", regexp_extract("raw_text", r"Amount\s+(.+?)\s+\d+\s+\$", 1))
    .withColumn("product_id", regexp_extract("raw_text", r"([A-Z]{3}-[A-Z]{2}-\d{4})", 1))
    .withColumn("category", regexp_extract("raw_text", r"(\bFurniture\b|\bOffice\s+Supplies\b|\bTechnology\b)", 1))
    .withColumn("sub_category", regexp_extract("raw_text", r"(\bChairs\b)", 1))
    .withColumn("quantity", regexp_extract("raw_text", r"Amount\s+.+?\s+(\d+)\s*\$", 1))
    .withColumn("unit_cost", regexp_extract("raw_text", r"Amount\s+.+?\s+\d+\s*\$(\d+\.\d+)", 1))
    .withColumn("subtotal", regexp_extract("raw_text", r"Subtotal:\s*\$?([\d,]+\.\d{2})", 1))
    .withColumn("discount", regexp_extract("raw_text", r"Discount.*\$\s*(\d+\.\d+)", 1))
    .withColumn("shipping_fee", regexp_extract("raw_text", r"Shipping:\s*\$(\d+\.\d+)", 1))
    .withColumn("total", regexp_extract("raw_text", r"Total:\s*\$?([\d,]+\.\d{2})", 1))
    .withColumn("order_id", regexp_extract("raw_text", r"Order ID\s*:\s*([A-Z]{2}-\d{4}-[A-Z0-9\-]+)", 1))
    
)

# Save structured data as Delta table (with schema merge)
parsed_df.write \
    .format("delta") \
    .option("mergeSchema", "true") \
    .mode("overwrite") \
    .save("/Volumes/databricks_catalog/invoice_schema/datatt")

# Save as a SQL table (with schema merge)
parsed_df.write \
    .format("delta") \
    .option("mergeSchema", "true") \
    .mode("overwrite") \
    .saveAsTable("databricks_catalog.invoice_schema.raw_inovice_table")

# View & Query the structured invoice
display(spark.sql("SELECT * FROM databricks_catalog.invoice_schema.raw_inovice_table"))